# 1. Сравнение двух средних


## 1.1 t-тест Стьюдента

### 1.1.1 Two-Sample T-Test (независимый, двухвыборочный t-тест Стьюдента)

In [1]:
# импорт библиотек
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

In [3]:
#считываем данные
df = pd.read_csv('../resources/lab1_3/marketing_eng.csv')
df.head()

,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,EDUCATION,MARITAL_STATUS,ORG_TP_STATE,JOB_DIR,PERSONAL_INCOME,OWN_AUTO,WORK_TIME,LOAN_MAX_DLQ_AMT
0,0,49,1,0,1,Среднее специальное,Состою в браке,Частная компания,Вспомогательный техперсонал,5000.0,0,18.0,1580.0
1,0,32,1,0,1,Среднее,Состою в браке,Индивидуальный предприниматель,Участие в основ. деятельности,12000.0,0,97.0,4020.0
2,0,52,1,0,1,Неполное среднее,Состою в браке,Государственная комп./учреж.,Участие в основ. деятельности,9000.0,0,84.0,0.0
3,0,39,1,0,1,Высшее,Состою в браке,Государственная комп./учреж.,Участие в основ. деятельности,25000.0,0,168.0,1590.0
4,0,30,1,0,0,Среднее,Состою в браке,Государственная комп./учреж.,Участие в основ. деятельности,12000.0,0,101.0,2230.0


In [4]:
#разделяем выборки по целевому критерию
age1 = df[df['TARGET'] == 1]
age2 = df[df['TARGET'] == 0]

In [5]:
# выводим средний возраст в каждой выборке
print(age1.AGE.mean(), age2.AGE.mean())

37.1280353200883 40.84900454850496


In [6]:
#осуществляем Two-Sample T-Test
stats.ttest_ind(a= age1.AGE, b= age2.AGE, equal_var=False)

TtestResult(statistic=np.float64(-14.202065231321146), pvalue=np.float64(4.4746367057134615e-44), df=np.float64(2488.2236039982467))

Согласно полученному значению P (pvalue=4.4746367057134615e-44), средние значения возраста у рассматриваемых групп клиентов статистически значимо различаются. Отвергая нулевую гипотезу о равенстве этих средних значений, мы рискуем ошибиться с вероятностью значительно менее 0.1%.

В то же время следует аккуратно прибегать к использованию этого теста! (https://habr.com/company/stepic/blog/250527/)

### 1.1.2 Paired T-Test (для связных выборок t-тест Стьюдента)

Данный тест служит для проверки эффективности какого-то мероприятия (например, показателей ОДНИХ И ТЕХ ЖЕ СПОРТСМЕНОВ ПОСЛЕ ДОПИНГА)

In [7]:
np.random.seed(11)

before= stats.norm.rvs(scale=30, loc=250, size=100)

after = before + stats.norm.rvs(scale=5, loc=-1.25, size=100)

weight_df = pd.DataFrame({"weight_before":before,
                          "weight_after":after,
                          "weight_change":after-before})

weight_df.describe()

,weight_before,weight_after,weight_change
count,100.000000,100.000000,100.000000
mean,250.345546,249.115171,-1.230375
std,28.132539,28.422183,4.783696
min,170.400443,165.913930,-11.495286
25%,230.421042,229.148236,-4.046211
50%,250.830805,251.134089,-1.413463
75%,270.637145,268.927258,1.738673
max,314.700233,316.720357,9.759282


Резюме показывает, что спортсмены потеряли около 1,23 кг в приседе в среднем после прима "Мельдоши". Давайте проведем парный t-тест, чтобы увидеть, является ли эта разница существенной с 95% -ным доверительным уровнем:

In [8]:
stats.ttest_rel(a = before, b = after)

TtestResult(statistic=np.float64(2.5720175998568284), pvalue=np.float64(0.011596444318439857), df=np.int64(99))

Значение p (pvalue=0.011596444318439857) в тестовом выходе показывает, что шансы увидеть полученную разницу (-1,23 кг) между выборками из-за случайности составляют чуть более 1%.

## 1.2. U-критерий Манна — Уитни

К сожалению, t-тест Стьюдента использует средние арифметические значения, что приводит к его неустойчивости при наличии аномальных выбросов и формы распределения, отличающейся от нормальной.

В такой ситуации прибегают к использованию U-критерия Манна-Уитни.
Он относится к непараметрическим критериям, и работает даже тогда, когда численные значения нам не известны (т.е. мы не знаем вес игрока сборной, а оперируем категориальными переменными – “худой”, “толстый”, “здоровенный” и т.д.)


In [9]:
stats.mannwhitneyu(x= age1.AGE, y= age2.AGE)

MannwhitneyuResult(statistic=np.float64(9963825.0), pvalue=np.float64(1.2818314854797137e-35))

# 2. Связь переменных между собой

## 2.1 Ковариация

Есть много путей посчитать ковариацию - первый, самый простой - это использовать фуекцияю Pandas .cov()

In [10]:
#считываем данные
new_df = pd.read_csv('/content/drive/MyDrive/Анализ данных и машинное обучение/Module2/data set 2.txt', sep='\t')

In [11]:
new_df.mean()

CREDIT    243483.258246
INCOME     29004.861565
dtype: float64

In [12]:
df.cov()

<ipython-input-12-6f98a29763d5>:1: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.cov()


,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,PERSONAL_INCOME,OWN_AUTO,WORK_TIME,LOAN_MAX_DLQ_AMT
TARGET,0.104869,-0.390215,0.007278,-0.009372,-0.004731,2.761141e+02,0.002181,-2.899051e+01,20.305012
AGE,-0.390215,134.584780,-1.507832,2.221635,0.785814,-9.406901e+03,-0.214045,-2.394454e+03,-370.252696
SOCSTATUS_WORK_FL,0.007278,-1.507832,0.082225,-0.078240,-0.013951,5.112420e+02,0.006706,9.772685e-02,8.720998
SOCSTATUS_PENS_FL,-0.009372,2.221635,-0.078240,0.116394,0.021427,-4.294663e+02,-0.008681,-6.738426e+00,-8.061541
GENDER,-0.004731,0.785814,-0.013951,0.021427,0.226134,-1.020874e+03,-0.040808,7.895260e+01,2.951569
PERSONAL_INCOME,276.114149,-9406.900511,511.241991,-429.466283,-1020.874322,8.127866e+07,516.975028,-1.098034e+06,568730.759542
OWN_AUTO,0.002181,-0.214045,0.006706,-0.008681,-0.040808,5.169750e+02,0.102941,-2.616620e+01,4.770274
WORK_TIME,-28.990509,-2394.453702,0.097727,-6.738426,78.952596,-1.098034e+06,-26.166201,5.936451e+08,-54285.614276
LOAN_MAX_DLQ_AMT,20.305012,-370.252696,8.720998,-8.061541,2.951569,5.687308e+05,4.770274,-5.428561e+04,699063.403699


## 2.2 Корреляция

In [11]:
#pandas
df.select_dtypes(include=[np.number]).corr()

,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,PERSONAL_INCOME,OWN_AUTO,WORK_TIME,LOAN_MAX_DLQ_AMT
TARGET,1.000000,-0.103868,0.078377,-0.084826,-0.030723,0.094575,0.020990,-0.003571,0.074993
AGE,-0.103868,1.000000,-0.453267,0.561319,0.142442,-0.089942,-0.057506,-0.009221,-0.038172
SOCSTATUS_WORK_FL,0.078377,-0.453267,1.000000,-0.799768,-0.102309,0.197759,0.072891,0.000157,0.036375
SOCSTATUS_PENS_FL,-0.084826,0.561319,-0.799768,1.000000,0.132074,-0.139629,-0.079307,-0.001280,-0.028261
GENDER,-0.030723,0.142442,-0.102309,0.132074,1.000000,-0.238123,-0.267468,0.006748,0.007424
PERSONAL_INCOME,0.094575,-0.089942,0.197759,-0.139629,-0.238123,1.000000,0.178726,-0.004964,0.075450
OWN_AUTO,0.020990,-0.057506,0.072891,-0.079307,-0.267468,0.178726,1.000000,-0.003259,0.017782
WORK_TIME,-0.003571,-0.009221,0.000157,-0.001280,0.006748,-0.004964,-0.003259,1.000000,-0.002591
LOAN_MAX_DLQ_AMT,0.074993,-0.038172,0.036375,-0.028261,0.007424,0.075450,0.017782,-0.002591,1.000000


In [14]:
#scipy
stats.pearsonr(new_df.INCOME, new_df.CREDIT)

PearsonRResult(statistic=0.10131925637391359, pvalue=2.0842849453322004e-111)

### 2.2.1. Корреляция для случаев аномалий в данных

В таких ситуациях прибегают к использованию

1) Коэффициент корреляции Спирмена

2) Коэффициент Кендалла (Кендалл-тау)

3) Другие виды ранговых корреляций


In [15]:
#pandas
new_df.corr(method='kendall')

,CREDIT,INCOME
CREDIT,1.000000,0.201172
INCOME,0.201172,1.000000


In [16]:
#pandas
df.corr(method='spearman')

<ipython-input-16-57acd8f57243>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr(method='spearman')


,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,PERSONAL_INCOME,OWN_AUTO,WORK_TIME,LOAN_MAX_DLQ_AMT
TARGET,1.000000,-0.100966,0.078377,-0.084826,-0.030723,0.100548,0.021074,-0.083306,0.079944
AGE,-0.100966,1.000000,-0.420150,0.522795,0.140331,-0.137494,-0.052014,0.420764,-0.050774
SOCSTATUS_WORK_FL,0.078377,-0.420150,1.000000,-0.799768,-0.102309,0.302420,0.072893,-0.005925,0.031059
SOCSTATUS_PENS_FL,-0.084826,0.522795,-0.799768,1.000000,0.132074,-0.213595,-0.079300,0.105921,-0.024478
GENDER,-0.030723,0.140331,-0.102309,0.132074,1.000000,-0.289067,-0.267436,0.076569,0.008780
PERSONAL_INCOME,0.100548,-0.137494,0.302420,-0.213595,-0.289067,1.000000,0.186508,0.045365,0.015158
OWN_AUTO,0.021074,-0.052014,0.072893,-0.079300,-0.267436,0.186508,1.000000,0.003873,0.007226
WORK_TIME,-0.083306,0.420764,-0.005925,0.105921,0.076569,0.045365,0.003873,1.000000,-0.036004
LOAN_MAX_DLQ_AMT,0.079944,-0.050774,0.031059,-0.024478,0.008780,0.015158,0.007226,-0.036004,1.000000


In [17]:
#scipy
stats.kendalltau(new_df.INCOME, new_df.CREDIT)

SignificanceResult(statistic=0.20117166371388148, pvalue=0.0)

In [18]:
#scipy
stats.spearmanr(new_df.INCOME, new_df.CREDIT)

SignificanceResult(statistic=0.28581899335770794, pvalue=0.0)

## 2.2.2 Тест хи-квадрат (Chi-Squared Test of Independence)

Тест на независимость обычно используется для определения того, изменяются ли такие переменные, как образование, политические взгляды и другие предпочтения, в зависимости от демографических факторов, таких как пол, раса и религия. Давайте создадим некоторые поддельные данные опроса избирателей и проведем проверку независимости:

In [19]:
np.random.seed(10)

# Образцы данных при фиксированных вероятностях
voter_race = np.random.choice(a= ["asian","black","hispanic","other","white"],
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5],
                              size=1000)

# Образцы данных при фиксированных вероятностях
voter_party = np.random.choice(a= ["democrat","independent","republican"],
                              p = [0.4, 0.2, 0.4],
                              size=1000)

voters = pd.DataFrame({"race":voter_race,
                       "party":voter_party})

voter_tab = pd.crosstab(voters.race, voters.party, margins = True)

voter_tab.columns = ["democrat","independent","republican","row_totals"]

voter_tab.index = ["asian","black","hispanic","other","white","col_totals"]

observed = voter_tab.iloc[0:5,0:3]   # Get table without totals for later use
voter_tab

,democrat,independent,republican,row_totals
asian,21,7,32,60
black,65,25,64,154
hispanic,107,50,94,251
other,15,8,15,38
white,189,96,212,497
col_totals,397,186,417,1000


In [20]:
voters

,race,party
0,white,democrat
1,asian,republican
2,white,independent
3,white,republican
4,other,democrat
...,...,...
995,white,republican
996,hispanic,independent
997,black,independent
998,white,republican


In [23]:
expected =  np.outer(voter_tab["row_totals"][0:5],
                     voter_tab.ix["col_totals"][0:3]) / 1000

expected = pd.DataFrame(expected)

expected.columns = ["democrat","independent","republican"]
expected.index = ["asian","black","hispanic","other","white"]

expected

KeyError: 'col_totals'

In [ ]:
stats.chi2_contingency(observed= observed)

In [ ]:
# то же но руками

chi_squared_stat = (((observed-expected)**2)/expected).sum().sum()

print(chi_squared_stat)

In [ ]:
crit = stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = 8)   # *

print("Critical value")
print(crit)

p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,  # Find the p-value
                             df=8)
print("P value")
print(p_value)

На выходе показаны статистика хи-квадрата, значение p и степени свободы, за которыми следуют ожидаемые подсчеты.
Как и ожидалось, с учетом высокого значения p результат теста не обнаруживает существенной взаимосвязи между переменными.

# 3. Факторный анализ (ANOVA)

In [ ]:
np.random.seed(12)

races =   ["asian","black","hispanic","other","white"]

# Генерировать случайные данные
voter_race = np.random.choice(a= races,
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5],
                              size=1000)

voter_age = stats.poisson.rvs(loc=18,
                              mu=30,
                              size=1000)

# Группируем возраст по расе
voter_frame = pd.DataFrame({"race":voter_race,"age":voter_age})
groups = voter_frame.groupby("race").groups

# Выделяем отдельные группы
asian = voter_age[groups["asian"]]
black = voter_age[groups["black"]]
hispanic = voter_age[groups["hispanic"]]
other = voter_age[groups["other"]]
white = voter_age[groups["white"]]

# Делаем ANOVA
stats.f_oneway(asian, black, hispanic, other, white)

А теперь сделаем наш результат значимым

In [ ]:
np.random.seed(12)

# Generate random data
voter_race = np.random.choice(a= races,
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5],
                              size=1000)

# Use a different distribution for white ages
white_ages = stats.poisson.rvs(loc=18,
                              mu=32,
                              size=1000)

voter_age = stats.poisson.rvs(loc=18,
                              mu=30,
                              size=1000)

voter_age = np.where(voter_race=="white", white_ages, voter_age)

# Group age data by race
voter_frame = pd.DataFrame({"race":voter_race,"age":voter_age})
groups = voter_frame.groupby("race").groups

# Extract individual groups
asian = voter_age[groups["asian"]]
black = voter_age[groups["black"]]
hispanic = voter_age[groups["hispanic"]]
other = voter_age[groups["other"]]
white = voter_age[groups["white"]]

# Perform the ANOVA
stats.f_oneway(asian, black, hispanic, other, white)

Выполним попарные сравнения

In [ ]:

race_pairs = []

for race1 in range(4):
    for race2  in range(race1+1):
        race_pairs.append((races[race1], races[race2]))


In [ ]:
race_pairs

In [ ]:

# Conduct t-test on each pair
for race1, race2 in race_pairs:
    print(race1, race2)
    print(stats.ttest_ind(voter_age[groups[race1]],
                          voter_age[groups[race2]]))

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(endog=voter_age,     # Данные
                          groups=voter_race,   # Группы
                          alpha=0.05)          # Уровень значимости

tukey.plot_simultaneous()    # доверительные интервалы
plt.vlines(x=49.57,ymin=-0.5,ymax=4.5, color="red")

tukey.summary()              # результат